# How to export your beautiful grid in a dataframe?

Now you have pivoted columns, aggregated, edited your data, it is time to export the data back to python. Remember the time when fitting your data from excel to python was a pain, inefficient and time-consuming ? Then you should definitely use the export features.

In [1]:
import os
import json
import pandas as pd
import numpy as np
import urllib.request as ur
from ipyaggrid import Grid


## Export simple data

Done only some filtering and sorting ? Export simple data from your grid. In auto-export mode, you can get the data you wand directly in dataframes as you click the grid.

In [2]:
def mklbl(prefix, n):
    return ["%s%s" % (prefix, i) for i in range(n)]

miindex = pd.MultiIndex.from_product([mklbl('A', 4),
                                      mklbl('B', 2),
                                      mklbl('C', 4),
                                      mklbl('D', 2)],
                                    names=['RowIdx-1', 'RowIdx-2', 'RowIdx-3', 'RowIdx-4'])
index  =['-'.join(col).strip() for col in miindex.values]
micolumns = pd.MultiIndex.from_tuples([('a', 'foo', 'zap'),
                                       ('a', 'foo', 'zip'),
                                       ('a', 'bar', 'zap'),
                                       ('a', 'bar', 'zip'),
                                       ('b', 'foo', 'zap'),
                                       ('b', 'foo', 'zep'),
                                       ('b', 'bah', 'zep'),
                                       ('b', 'bah', 'zyp'),
                                       ('b', 'bah', 'zap'),
                                      ],
                                      names=['ColIdx-{}'.format(i) for i in range(1, 4)])
cols  =['-'.join(col).strip() for col in micolumns.values]
data = np.arange(len(miindex) * len(micolumns), dtype=np.float64).reshape((len(miindex),len(micolumns)))
data = data.tolist()
df  = pd.DataFrame(data, index=index, columns=cols).sort_index()
df  = pd.DataFrame(data, index=index, columns=cols).sort_index()
df.index.name = 'UniqueRow'
df.columns.name = 'UniqueCol'

column_defs_1 = [{'field': df.index.name}] + [{'field': c} for c in df.columns[:5]]


grid_options_1 = {
    'columnDefs' : column_defs_1,
    'defaultColDef': {'sortable': 'true', 'filter': 'true', 'resizable': 'true'},
    'enableRangeSelection': 'true',
    'rowSelection': 'multiple',
}

grid1 = Grid(grid_data=df,
            grid_options=grid_options_1,
            quick_filter=True,
            export_csv=False,
            export_excel=False,
            #show_toggle_edit=True,
            export_mode='auto',
            index=True,
            keep_multiindex=False,
            theme='ag-theme-fresh')
grid1

Grid(columns_fit='size_to_fit', compress_data=True, export_mode='auto', height='350px', menu={'buttons': [], '…

Now try to sort, filter and select specific parts of the grid with the range selection and observe the value of grid_data_out.

In [3]:
# grid1.grid_data_out.get('grid')
# grid1.grid_data_out.get('rows')
#grid1.grid_data_out.get('cols')
grid1.grid_data_out.get('range')

,a-foo-zap,a-foo-zip,a-bar-zap
Index,,,
3,27,28,29
4,36,37,38
5,45,46,47
6,54,55,56


With other options like row selection, we get another way of exporting the data. Please try to remove columns, selecting several lines and observe the export values.

In [4]:
column_defs_2 = [{'field': df.index.name}] + [{'field': c} for c in df.columns[:5]]

grid_options_2 = {
    'columnDefs' : column_defs_2,
    'rowSelection': 'multiple',
}

grid2 = Grid(grid_data=df,
            grid_options=grid_options_2,
            quick_filter=True,
            export_csv=False,
            export_excel=False,
            show_toggle_edit=True,
            export_mode='buttons',
            index=True,
            keep_multiindex=False,
            theme='ag-theme-fresh')
grid2

Grid(columns_fit='size_to_fit', compress_data=True, export_mode='buttons', height='350px', menu={'buttons': [{…

In [6]:
# grid2.grid_data_out.get('grid')
grid2.grid_data_out.get('rows')

,UniqueRow,a-foo-zap,a-foo-zip,a-bar-zap,a-bar-zip,b-foo-zap
Index,,,,,,
5,A0-B0-C2-D1,45,46,47,48,49
6,A0-B0-C3-D0,54,55,56,57,58
7,A0-B0-C3-D1,63,64,65,66,67


## Exporting more complex grids

Now let's try to export more complex grids. With huge/complex data, we often want to export the data "as is". The export grid feature is available with the `button-export` mode.

In [7]:
url = 'https://raw.githubusercontent.com/widgetti/ipyaggrid/master/data/OlympicWinners.json'
with ur.urlopen(url) as res:
    data = json.loads(res.read().decode('utf-8'))
    

In [8]:
columnDefs = [
    {'headerName': "Country", 'field': "country", 'width': 120, 'rowGroup': 'true'},
    {'headerName': "Year", 'field': "year", 'width': 90, 'pivot': 'true'},
    {'headerName': "Sport", 'field': "sport", 'width': 110, 'rowGroup': 'true'},
    {'headerName': "Athlete", 'field': "athlete"},
    {'headerName': "Gold", 'field': "gold", 'width': 100, 'aggFunc': 'sum', 'sortable':True},
    {'headerName': "Silver", 'field': "silver", 'width': 100, 'aggFunc': 'sum'},
    {'headerName': "Bronze", 'field': "bronze", 'width': 100, 'aggFunc': 'sum'},
];

gridOptions = {
    'pivotMode': 'true',
    'defaultColDef': {'sortable': 'true', 'filter': 'true', 'resizable': 'true'},
    'columnDefs': columnDefs,
    'animateRows':'true',
};

pivot = Grid(export_mode='auto',
             export_csv=True,
             export_excel=True,
             quick_filter=True,
             theme='ag-theme-fresh',
             compress_data=True,
             grid_options=gridOptions,
             grid_data=data,
             columns_fit='auto',
             hide_grid=False)
pivot

Grid(columns_fit='auto', compress_data=True, export_mode='auto', height='350px', menu={'buttons': [{'name': 'E…

You could of course export the grid to CSV or Excel if you're a bit old fashioned, but an option to consider is definitely to export it directly to python via `grid_data_out`.

You see that the data in the first column has been grouped several times. You can thus choose to display the aggregated data at the country level or at the sport level by incrementing the counter next to the *Export Grid* button. Then try to export the grid by clicking the button `Export Grid` and observe the result.

In [10]:
pivot.grid_data_out['grid']

2000                2002                 2004                \
                Gold Silver Bronze  Gold Silver Bronze   Gold Silver Bronze   
country                                                                       
United States  130.0   61.0   52.0  11.0   58.0   15.0  118.0   75.0   72.0   
Russia          66.0   67.0   54.0   6.0    5.0   27.0   47.0   47.0   97.0   
Australia       60.0   69.0   54.0   2.0    0.0    0.0   49.0   77.0   30.0   
Canada           4.0    4.0   23.0  52.0    7.0   15.0    3.0   10.0    4.0   
Norway          20.0    4.0   19.0  23.0   11.0    7.0    5.0    0.0    2.0   
...              ...    ...    ...   ...    ...    ...    ...    ...    ...   
Panama           NaN    NaN    NaN   NaN    NaN    NaN    NaN    NaN    NaN   
Eritrea          NaN    NaN    NaN   NaN    NaN    NaN    0.0    0.0    1.0   
Sri Lanka        0.0    1.0    0.0   NaN    NaN    NaN    NaN    NaN    NaN   
Mozambique       1.0    0.0    0.0   NaN    NaN    NaN    NaN    NaN    NaN   
Barbados         0.0    0.0    1.0   NaN    NaN    NaN    NaN    NaN    NaN   

               2006  ...          2008                2010                \
               Gold  ... Bronze   Gold Silver Bronze  Gold Silver Bronze   
country              ...                                                   
United States   9.0  ...   32.0  127.0  109.0   81.0  12.0   63.0   22.0   
Russia         16.0  ...   13.0   43.0   46.0   57.0   6.0    5.0   14.0   
Australia       1.0  ...    1.0   31.0   42.0   76.0   2.0    1.0    0.0   
Canada         30.0  ...   11.0   11.0   13.0   10.0  67.0   15.0    8.0   
Norway          2.0  ...   12.0   16.0    5.0    1.0  16.0   14.0    9.0   
...             ...  ...    ...    ...    ...    ...   ...    ...    ...   
Panama          NaN  ...    NaN    1.0    0.0    0.0   NaN    NaN    NaN   
Eritrea         NaN  ...    NaN    NaN    NaN    NaN   NaN    NaN    NaN   
Sri Lanka       NaN  ...    NaN    NaN    NaN    NaN   NaN    NaN    NaN   
Mozambique      NaN  ...    NaN    NaN    NaN    NaN   NaN    NaN    NaN   
Barbados        NaN  ...    NaN    NaN    NaN    NaN   NaN    NaN    NaN   

                2012                
                Gold Silver Bronze  
country                             
United States  145.0   63.0   46.0  
Russia          50.0   39.0   51.0  
Australia       18.0   37.0   59.0  
Canada           1.0   21.0   33.0  
Norway          15.0    1.0    1.0  
...              ...    ...    ...  
Panama           NaN    NaN    NaN  
Eritrea          NaN    NaN    NaN  
Sri Lanka        NaN    NaN    NaN  
Mozambique       NaN    NaN    NaN  
Barbados         NaN    NaN    NaN  

[110 rows x 21 columns]